In [291]:
#!pip install spacy[cuda90]

from __future__ import unicode_literals, print_function
import pandas as pd
import spacy
import pickle
import plac
import random
from pathlib import Path
from spacy.util import minibatch, compounding
import plac
import logging
import argparse
import sys
import os
import json
import nltk
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn_crfsuite import CRF, scorers, metrics
from sklearn_crfsuite.metrics import flat_classification_report
from sklearn.metrics import classification_report
from sklearn.metrics import classification_report, make_scorer
from sklearn.model_selection import RandomizedSearchCV
import scipy.stats
import eli5
import warnings
from tqdm import tqdm
import pickle
import joblib
import numpy as np
tqdm.pandas()
warnings.filterwarnings('ignore')

In [4]:
train=pd.read_csv('Hackathon-diseases/train_data.csv')
test=pd.read_csv('Hackathon-diseases/test_data.csv')

train=train.dropna()

In [5]:
train['tag'].value_counts()

O                115955
B-indications      1501
I-indications      1234
Name: tag, dtype: int64

### Q1. Spacy's custom training pipeline

In [6]:

ttag=train[['Word','tag']]

#save it in tsv format
with open('Hackathon-diseases/train.tsv','w') as write_tsv:
    write_tsv.write(ttag.to_csv(sep='\t', index=False,header=None))

In [7]:
def tsv_to_json_format(input_path,output_path,unknown_label):
    try:
        f=open(input_path,'r') # input file
        fp=open(output_path, 'w') # output file
        data_dict={}
        annotations =[]
        label_dict={}
        s=''
        start=0
        for line in f:
            if line[0:len(line)-1]!='.\tO':
                word,entity=line.split('\t')
                s+=word+" "
                entity=entity[:len(entity)-1]
                if entity!=unknown_label:
                    if len(entity) != 1:
                        d={}
                        d['text']=word
                        d['start']=start
                        d['end']=start+len(word)-1  
                        try:
                            label_dict[entity].append(d)
                        except:
                            label_dict[entity]=[]
                            label_dict[entity].append(d) 
                start+=len(word)+1
            else:
                data_dict['content']=s
                s=''
                label_list=[]
                for ents in list(label_dict.keys()):
                    for i in range(len(label_dict[ents])):
                        if(label_dict[ents][i]['text']!=''):
                            l=[ents,label_dict[ents][i]]
                            for j in range(i+1,len(label_dict[ents])): 
                                if(label_dict[ents][i]['text']==label_dict[ents][j]['text']):  
                                    di={}
                                    di['start']=label_dict[ents][j]['start']
                                    di['end']=label_dict[ents][j]['end']
                                    di['text']=label_dict[ents][i]['text']
                                    l.append(di)
                                    label_dict[ents][j]['text']=''
                            label_list.append(l)                          
                            
                for entities in label_list:
                    label={}
                    label['label']=[entities[0]]
                    label['points']=entities[1:]
                    annotations.append(label)
                data_dict['annotation']=annotations
                annotations=[]
                json.dump(data_dict, fp)
                fp.write('\n')
                data_dict={}
                start=0
                label_dict={}
    except Exception as e:
        logging.exception("Unable to process file" + "\n" + "error = " + str(e))
        return None

In [8]:
#Converting it into json file for spacy training format
tsv_to_json_format("Hackathon-diseases/train.tsv",'Hackathon-diseases/train.json','abc')

In [110]:
# Convert json file to spaCy format.

def main_format(input_file=None, output_file=None):
    try:
        training_data = []
        lines=[]
        with open(input_file, 'r') as f:
            lines = f.readlines()

        for line in lines:
            data = json.loads(line)
            text = data['content']
            entities = []
            for annotation in data['annotation']:
                point = annotation['points'][0]
                labels = annotation['label']
                if not isinstance(labels, list):
                    labels = [labels]

                for label in labels:
                    entities.append((point['start'], point['end'] + 1 ,label))


            training_data.append((text, {"entities" : entities}))

        with open(output_file, 'wb') as fp:
            pickle.dump(training_data, fp)

    except Exception as e:
        logging.exception("Unable to process " + input_file + "\n" + "error = " + str(e))
        return None

In [10]:
main_format('Hackathon-diseases/train.json','Hackathon-diseases/train')

In [146]:
# Specify the new entity labels which you want to add here
LABEL = ['B-indications', 'I-indications','O']


# Loading training data 
with open ('Hackathon-diseases/train', 'rb') as fp:
    TRAIN_DATA = pickle.load(fp)


def main(model=None,traindata=[],new_model_name='new_model', output_dir=None,labels=[],n_iter=10):
    """Setting up the pipeline and entity recognizer, and training the new entity."""
    if model is not None:
        nlp = spacy.load(model)  # load existing spacy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank('en')  # create blank Language class
        print("Created blank 'en' model")
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner)
    else:
        ner = nlp.get_pipe('ner')

    for i in labels:
        ner.add_label(i)   # Add new entity labels to entity recognizer

    if model is None:
        optimizer = nlp.begin_training()
    else:
        optimizer = nlp.entity.create_optimizer()

    # Get names of other pipes to disable them during training to train only NER
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        for itn in range(n_iter):
            random.shuffle(traindata)
            losses = {}
            batches = minibatch(traindata, size=compounding(4., 32., 1.001))
            
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(texts, annotations, sgd=optimizer, drop=0.2,
                           losses=losses)
                
            if itn%50==0:
                print(f'Losses for iteration: {itn}==>', losses)
            
    # Test the trained model
    test_text = 'Obesity as a concomitant cause in the complex etiology of arteriosclerosis'
    doc = nlp(test_text)
    print("Entities in '%s'" % test_text)
    for ent in doc.ents:
        print(ent.label_, ent.text)

    # Save model 
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.meta['name'] = new_model_name  # rename model
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

        # Test the saved model
        print("Loading from", output_dir)
        nlp2 = spacy.load(output_dir)
        doc2 = nlp2(test_text)
        for ent in doc2.ents:
            print(ent.label_, ent.text)

In [19]:
#just training for 100 iteration.Longer training will give better results

spacy.prefer_gpu()
main('en_core_web_md',traindata=TRAIN_DATA,output_dir='Hackathon-diseases/',labels=LABEL,n_iter=100)

Loaded model 'en_core_web_md'
Losses for iteration: 0==> {'ner': 43479.45319137641}
Losses for iteration: 50==> {'ner': 39665.00170284374}
Entities in 'Obesity as a concomitant cause in the complex etiology of arteriosclerosis'
B-indications arteriosclerosis
Saved model to Hackathon-diseases
Loading from Hackathon-diseases
B-indications arteriosclerosis


### Q2. Catastrophic forgetting

The problem when finetuning on custom dataset is, the existing learnt information space might be lost/overwritten. This is otherwise known as catastrophic forgetting. 

To counter this problem, we can usually do pseudo labelling/rehearsal.

All it does is the following:
1)Once the model is fine tuned on a specific dataset, we can take most confident predictions from the
trained model
2)Mix it up with old data and train the ner model again with mix of both of these data.

In this way, the model get to see the best of both worlds, thereby retaining the information.

In [134]:
conlldata=pd.read_csv('ner_dataset.csv',encoding="ISO-8859-1")
tags=['B-per','I-per','B-org','I-org','O']
conlldata=conlldata[:4403]
conlldata[conlldata['Tag'].isin(tags)]['Tag'].value_counts()

O        3778
B-org      96
I-per      71
I-org      68
B-per      57
Name: Tag, dtype: int64

In [105]:
#predict some sample set from test data using the above trained model and add it into the conlldata
def get_tags(nlp,text):
    doc=nlp(text)
    keydict={}
    tags=[]
    
    for ent in doc.ents:
        keydict[ent.text]=ent.label_

    for word in nltk.word_tokenize(text):
        if word not in keydict.keys():
            tags.append('O')
        else:
            tags.append(keydict[word])

    return tags

        
sample_test=test[-2000:]
sample_test=sample_test.groupby('Sent_ID')['Word'].apply(list).reset_index()
sample_test['sent']=sample_test['Word'].apply(lambda x:' '.join(x))
nlp = spacy.load('Hackathon-diseases/')

sample_test['Tag']=sample_test['sent'].progress_apply(lambda x: get_tags(nlp,x))

100%|██████████| 79/79 [00:00<00:00, 132.43it/s]


In [106]:
def explode(df, lst_cols, fill_value=''):
    # make sure `lst_cols` is a list
    if lst_cols and not isinstance(lst_cols, list):
        lst_cols = [lst_cols]
    # all columns except `lst_cols`
    idx_cols = df.columns.difference(lst_cols)

    # calculate lengths of lists
    lens = df[lst_cols[0]].str.len()

    if (lens > 0).all():
        # ALL lists in cells aren't empty
        return pd.DataFrame({
            col:np.repeat(df[col].values, df[lst_cols[0]].str.len())
            for col in idx_cols
        }).assign(**{col:np.concatenate(df[col].values) for col in lst_cols}) \
          .loc[:, df.columns]
    else:
        # at least one list in cells is empty
        return pd.DataFrame({
            col:np.repeat(df[col].values, df[lst_cols[0]].str.len())
            for col in idx_cols
        }).assign(**{col:np.concatenate(df[col].values) for col in lst_cols}) \
          .append(df.loc[lens==0, idx_cols]).fillna(fill_value) \
          .loc[:, df.columns]
    
sample_test=explode(sample_test, lst_cols=['Word','Tag'])

In [107]:
sample_test=sample_test[['Word','Tag']]
sample_test.head()

,Word,Tag
0,blindness,O
1,rate,O
2,of,O
3,POAG,B-indications
4,was,O


In [135]:
conlldatasample=conlldata[['Word','Tag']]

newdata=sample_test.append(conlldatasample)

#save it in tsv format
with open('Hackathon-diseases/new_train.tsv','w') as write_tsv:
    write_tsv.write(newdata.to_csv(sep='\t', index=False,header=None))
    

#Converting it into json file for spacy training format
tsv_to_json_format("Hackathon-diseases/new_train.tsv",'Hackathon-diseases/new_train.json','abc')

main_format('Hackathon-diseases/new_train.json','Hackathon-diseases/new_train')

In [144]:
# Loading training data 
with open ('Hackathon-diseases/new_train', 'rb') as fp:
    TRAIN_DATA = pickle.load(fp)

In [147]:
spacy.prefer_gpu()
main('Hackathon-diseases/',traindata=TRAIN_DATA,output_dir='cfmodel/',new_model_name='newmodel_cf',n_iter=30)

Loaded model 'Hackathon-diseases/'
Losses for iteration: 0==> {'ner': 489.93661540223263}
Entities in 'Obesity as a concomitant cause in the complex etiology of arteriosclerosis'
B-indications arteriosclerosis
Saved model to cfmodel
Loading from cfmodel
B-indications arteriosclerosis


### Q3. CRF

In [177]:
#Creating pos tags for additional features into crf model
train['POS']=train['Word'].apply(lambda x: nltk.pos_tag([x])[0][1])

In [185]:
train.head(3)

,Unnamed: 0,id,Doc_ID,Sent_ID,Word,tag,POS
0,0,1,1,1,Obesity,O,NN
1,1,2,1,1,in,O,IN
2,2,3,1,1,Low-,O,JJ


In [179]:
# A class to retrieve the sentences from the dataset
class getsentence(object):
    
    def __init__(self, data):
        self.n_sent = 1.0
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["tag"].values.tolist())]
        self.grouped = self.data.groupby("Doc_ID").apply(agg_func)
        self.sentences = [s for s in self.grouped]

In [186]:
getter = getsentence(train)
sentences = getter.sentences

#This is how a sentence will look like. 
print(sentences[0])

[('Obesity', 'NN', 'O'), ('in', 'IN', 'O'), ('Low-', 'JJ', 'O'), ('and', 'CC', 'O'), ('Middle-Income', 'JJ', 'O'), ('Countries', 'NNS', 'O'), (':', ':', 'O'), ('Burden', 'NN', 'O'), (',', ',', 'O'), ('Drivers', 'NNS', 'O'), (',', ',', 'O'), ('and', 'CC', 'O'), ('Emerging', 'VBG', 'O'), ('Challenges', 'NNS', 'O'), ('.', '.', 'O'), ('We', 'PRP', 'O'), ('have', 'VB', 'O'), ('reviewed', 'VBN', 'O'), ('the', 'DT', 'O'), ('distinctive', 'NN', 'O'), ('features', 'NNS', 'O'), ('of', 'IN', 'O'), ('excess', 'NN', 'O'), ('weight', 'NN', 'O'), (',', ',', 'O'), ('its', 'PRP$', 'O'), ('causes', 'NNS', 'O'), (',', ',', 'O'), ('and', 'CC', 'O'), ('related', 'JJ', 'O'), ('prevention', 'NN', 'O'), ('and', 'CC', 'O'), ('management', 'NN', 'O'), ('efforts', 'NNS', 'O'), (',', ',', 'O'), ('as', 'IN', 'O'), ('well', 'RB', 'O'), ('as', 'IN', 'O'), ('data', 'NNS', 'O'), ('gaps', 'NNS', 'O'), ('and', 'CC', 'O'), ('recommendations', 'NNS', 'O'), ('for', 'IN', 'O'), ('future', 'NN', 'O'), ('research', 'NN', 'O')

In [154]:
# Feature set
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features

In [155]:
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

In [156]:
#Creating the train and test set
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

In [157]:
#Creating the CRF model
crf = CRF(algorithm='lbfgs',
          c1=0.1,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False)

In [158]:
#We predict using the same 5 fold cross validation
pred = cross_val_predict(estimator=crf, X=X, y=y, cv=5)

In [159]:
#Lets evaluate the model
report = flat_classification_report(y_pred=pred, y_true=y)
print(report)

               precision    recall  f1-score   support

B-indications       0.72      0.35      0.47      1501
I-indications       0.76      0.41      0.53      1234
            O       0.99      1.00      0.99    115955

     accuracy                           0.98    118690
    macro avg       0.82      0.58      0.66    118690
 weighted avg       0.98      0.98      0.98    118690



In [160]:
%%time
#Now we will create the Randomized CV search model wherein we will use a modified F1 scorer model considering only the relevant labels
# define fixed parameters and parameters to search

params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=['B-indications','I-indications'])

# search
rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=50,
                        scoring=f1_scorer)
rs.fit(X, y)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 20.1min finished


CPU times: user 20min 3s, sys: 2.32 s, total: 20min 6s
Wall time: 20min 14s


RandomizedSearchCV(cv=3,
                   estimator=CRF(algorithm='lbfgs',
                                 all_possible_transitions=False, c1=0.1, c2=0.1,
                                 keep_tempfiles=None, max_iterations=100),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'c1': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fb366dd3da0>,
                                        'c2': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fb366dd3eb8>},
                   scoring=make_scorer(flat_f1_score, average=weighted, labels=['B-indications', 'I-indications']),
                   verbose=1)

In [161]:
#Lets check the best estimated parameters and CV score
print('Best parameters:', rs.best_params_)
print('Best CV score:', rs.best_score_)
print('Model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

Best parameters: {'c1': 0.21751174590712863, 'c2': 0.006721452694285955}
Best CV score: 0.5083597891934707
Model size: 0.26M


In [162]:
#We sort the tags a bit so that they appear in an orderly fashion in the classification report
sorted_labels = sorted(
    ['B-indications','I-indications'],
    key=lambda name: (name[1:], name[0]))

In [163]:
#Now we create the model again using the best estimators
crf3 = rs.best_estimator_
y_pred = crf3.predict(X)
print(metrics.flat_classification_report(
    y, y_pred, labels=sorted_labels, digits=3))

               precision    recall  f1-score   support

B-indications      0.992     0.985     0.988      1501
I-indications      0.980     0.987     0.983      1234

    micro avg      0.986     0.986     0.986      2735
    macro avg      0.986     0.986     0.986      2735
 weighted avg      0.987     0.986     0.986      2735



In [187]:
#Creating the CRF model
crf = CRF(algorithm='lbfgs',
          c1=0.21751174590712863,
          c2=0.006721452694285955,
          max_iterations=100,
          all_possible_transitions=False)

crf.fit(X,y)

CRF(algorithm='lbfgs', all_possible_transitions=False, c1=0.21751174590712863,
    c2=0.006721452694285955, keep_tempfiles=None, max_iterations=100)

In [188]:
y_pred = crf.predict(X)
print(metrics.flat_classification_report(
    y, y_pred,digits=3))

               precision    recall  f1-score   support

B-indications      0.992     0.985     0.988      1501
I-indications      0.980     0.987     0.983      1234
            O      1.000     1.000     1.000    115955

     accuracy                          0.999    118690
    macro avg      0.991     0.990     0.991    118690
 weighted avg      0.999     0.999     0.999    118690



In [166]:
eli5.show_weights(crf3, top=30)

In [167]:
# Save to file in the current working directory
joblib_file = "crf.pkl"
joblib.dump(crf3, joblib_file)

['crf.pkl']

### Test data to compare both models

In [189]:
test.head(3)

,Unnamed: 0,id,Doc_ID,Sent_ID,Word,tag,POS
0,0,0,1,0,Current,O,NN
1,1,1,1,0,definition,O,NN
2,2,2,1,0,and,O,CC


In [190]:
###CRF model output for the validation data

#Creating pos tags for additional features into crf model
test['POS']=test['Word'].apply(lambda x: nltk.pos_tag([x])[0][1])

getter = getsentence(test)
sentences = getter.sentences

#This is how a sentence will look like. 
print(sentences[0])

[('Current', 'NN', 'O'), ('definition', 'NN', 'O'), ('and', 'CC', 'O'), ('treatment', 'NN', 'O'), ('of', 'IN', 'O'), ('premature', 'NN', 'B-indications'), ('ejaculation', 'NN', 'I-indications'), ('.', '.', 'O'), ('To', 'TO', 'O'), ('discuss', 'NN', 'O'), ('a', 'DT', 'O'), ('concept', 'NN', 'O'), ('about', 'IN', 'O'), ('the', 'DT', 'O'), ('current', 'JJ', 'O'), ('definition', 'NN', 'O'), ('and', 'CC', 'O'), ('treatment', 'NN', 'O'), ('of', 'IN', 'O'), ('premature', 'NN', 'B-indications'), ('ejaculation', 'NN', 'I-indications'), ('.', '.', 'O'), ('A', 'DT', 'O'), ('detailed', 'JJ', 'O'), ('review', 'NN', 'O'), ('was', 'VBD', 'O'), ('performed', 'VBN', 'O'), ('on', 'IN', 'O'), ('the', 'DT', 'O'), ('current', 'JJ', 'O'), ('definition', 'NN', 'O'), ('and', 'CC', 'O'), ('treatment', 'NN', 'O'), ('of', 'IN', 'O'), ('premature', 'NN', 'B-indications'), ('ejaculation', 'NN', 'I-indications'), ('.', '.', 'O'), ('Several', 'JJ', 'O'), ('definitions', 'NNS', 'O'), ('were', 'VBD', 'O'), ('found', '

In [268]:
#validation data
valid_X = [sent2features(s) for s in sentences]
valid_y = [sent2labels(s) for s in sentences]

# Load from file
crf_model = joblib.load(joblib_file)

Ypredict = crf_model.predict(valid_X)

print(metrics.flat_classification_report(
    valid_y, Ypredict,digits=3))

               precision    recall  f1-score   support

B-indications      0.704     0.393     0.505       351
I-indications      0.742     0.487     0.588       277
            O      0.990     0.998     0.994     30678

     accuracy                          0.987     31306
    macro avg      0.812     0.626     0.696     31306
 weighted avg      0.984     0.987     0.985     31306



In [247]:
####Loading spacy ner model

spacyner = spacy.load('Hackathon-diseases/')

def get_tags(nlp,text):
    doc=nlp(text)
    keydict={}
    tags=[]
    
    for ent in doc.ents:
        keydict[ent.text]=ent.label_

    for word in text.split(' '):
        if word not in keydict.keys():
            tags.append('O')
        else:
            tags.append(keydict[word])

    return tags

In [248]:
sample_test=test.groupby('Sent_ID')['Word'].apply(list).reset_index()
sample_test['sent']=sample_test['Word'].apply(lambda x:' '.join(x))

sample_test['spacy_tag']=sample_test['sent'].progress_apply(lambda x: get_tags(spacyner,x))

# test['spacy_tag']=test['Word'].progress_apply(lambda x: get_tags(spacyner,x))

100%|██████████| 1291/1291 [00:11<00:00, 111.75it/s]


In [249]:
sample_test=explode(sample_test, lst_cols=['Word','spacy_tag'])
sample_test=sample_test[['Word','spacy_tag']]

In [292]:
y_pred=sample_test['spacy_tag'].values
valid_y=test['tag'].values

print(classification_report(
    valid_y, y_pred,digits=3))

               precision    recall  f1-score   support

B-indications      0.556     0.470     0.509       351
I-indications      0.528     0.585     0.555       277
            O      0.991     0.992     0.992     30678

     accuracy                          0.983     31306
    macro avg      0.692     0.682     0.685     31306
 weighted avg      0.982     0.983     0.983     31306



Both seems to be performing with same score. But there is a lot of scope for improvements in both models